### URL parsing

In [190]:
import urllib.parse

# 한글 문자열
text = "강남역 맛집"

# URL 인코딩
encoded_text = urllib.parse.quote(text)

base_path=f"https://www.google.co.kr/maps/search/{encoded_text}"

print(encoded_text)

%EA%B0%95%EB%82%A8%EC%97%AD%20%EB%A7%9B%EC%A7%91


### googlemap_info

In [2]:
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.expected_conditions import visibility_of_all_elements_located,presence_of_element_located,presence_of_all_elements_located

# 레스토랑 db insert 기초 틀
def restuarant_properties(search_term,restaurant_name,url,address,rating,category,**kwrargs):
    
    properties={"검색어" : {"rich_text" : [{"text": {"content": search_term}}]},
                "음식점이름" : {"title" : [{"text" : {'content': restaurant_name}}]},
                "URL" : {"url" : url},
                "주소" : {"rich_text" : [{"text": {"content": address}}]},
                "별점" : {"number" : rating},
                "카테고리" : {"rich_text" : [{"text": {"content": category}}]}
                }
    return properties

# 리뷰 db insert 기초 틀
def review_properties(restaurant_name,user,review,**kwargs):

    properties={"음식점이름" : {"title" : [{"text": {"content": restaurant_name}}]},
                "유저" : {"rich_text" : [{"text": {"content": user}}]},
                "리뷰" : {"rich_text" : [{"text": {"content": review}}]}
                }
    return properties


info_num=20

Main_Drive=Chrome()
Main_Drive.get(base_path)
Sub_Drive=Chrome()

main_box=WebDriverWait(Main_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="feed"]']))

while True:
    main_box.send_keys(Keys.PAGE_DOWN)

    feed_box=WebDriverWait(Main_Drive,3).until(
        visibility_of_all_elements_located([By.CLASS_NAME,"hfpxzc"]))
    
    if len(feed_box)>info_num:
        break

infos=[]

for sub_feed in feed_box:
    search_term=text #위의 코드 에서 가져옴 수정필요
    restaurant_name=sub_feed.get_attribute("aria-label")
    url=sub_feed.get_attribute("href")

    infos.append(
        restuarant_properties(search_term,restaurant_name,url,None,None,None)
    )

for sub_info in infos:
    Sub_Drive.get(sub_info["URL"])

    #별점
    rating=WebDriverWait(Sub_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="img"]'])).get_attribute("aria-label")
    # 전처리해야함
    rating="".join([term for term in rating if not term.isalpha()]).strip()
    rating=float(rating)

    #음식점 카테고리
    category=WebDriverWait(Sub_Drive,3).until(
    presence_of_element_located([By.CSS_SELECTOR,'button.DkEaL'])).text

    #정보선택칸
    tabs=WebDriverWait(Sub_Drive,3).until(
    presence_of_all_elements_located([By.XPATH, '//*[@role="tab"]']))
    tab_select={element.text:element for element in tabs}
    
    #하위 정보칸
    info_region=WebDriverWait(Sub_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="region"]']))
    #주소
    address=WebDriverWait(info_region,3).until(
    presence_of_element_located([By.XPATH, '//*[@data-item-id="address"]']))
    

/Users/ijeonghun/Documents/github/GoogleMap/googlemap/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [105]:
Main_Drive=Chrome()
Main_Drive.get(base_path)
info_num=30
#Sub_Drive=Chrome()

main_box=WebDriverWait(Main_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="feed"]']))

while True:
    main_box.send_keys(Keys.PAGE_DOWN)

    feed_box=WebDriverWait(Main_Drive,3).until(
        visibility_of_all_elements_located([By.CSS_SELECTOR,"a.hfpxzc"]))
    
    if len(feed_box)>info_num:
        break

infos=[]

for sub_feed in feed_box:
    search_term=text #위의 코드 에서 가져옴 수정필요
    restaurant_name=sub_feed.get_attribute("aria-label")
    url=sub_feed.get_attribute("href")

    infos.append(
        restuarant_properties(search_term,restaurant_name,url,None,None,None)
    )

In [107]:
Main_Drive.quit()

In [135]:
from threading import Thread

In [184]:
#%%writefile GooglemapCraw.py
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import urllib.parse
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.expected_conditions import visibility_of_all_elements_located,presence_of_element_located,presence_of_all_elements_located
from threading import Thread
from typing import List,Dict

class GooglemapCraw():
    def  __init__(self,search_term,craw_num:int,review_num:int=100,thread_num:int=1,optional:bool=True):

        # Chrome 옵션 설정
        self.chrome_options = Options()
        if optional:
            self.chrome_options.add_argument("--headless")  # Headless 모드
        else:
            pass

        self.main_drive=Chrome(options=self.chrome_options) # url까지 크롤
        self.craw_num=craw_num
        self.search_term=search_term
        self.infos=[] # 레스토랑 저장공간
        self.reviews=[] #리뷰 저장공간
        self.thread_num=thread_num
        self.review_num=review_num

        encoded_text = urllib.parse.quote(self.search_term)
        self.base_path=f"https://www.google.co.kr/maps/search/{encoded_text}" # 크롤링이 될 주소(파싱될 예정)
        pass

    def __restuarant_properties(self,search_term,restaurant_name,url,address,rating,category,**kwrargs):
    
        properties={"검색어" : {"rich_text" : [{"text": {"content": search_term}}]},
                "음식점이름" : {"title" : [{"text" : {'content': restaurant_name}}]},
                "URL" : {"url" : url},
                "주소" : {"rich_text" : [{"text": {"content": address}}]},
                "별점" : {"number" : rating},
                "카테고리" : {"rich_text" : [{"text": {"content": category}}]}
                }
        return properties

    # 리뷰 db insert 기초 틀
    def __review_properties(self,restaurant_name,user,review,**kwargs):

        properties={"음식점이름" : {"title" : [{"text": {"content": restaurant_name}}]},
                    "유저" : {"rich_text" : [{"text": {"content": user}}]},
                    "리뷰" : {"rich_text" : [{"text": {"content": review}}]}
                    }
        return properties
    
    # 멀티 스래딩 적용 + 리뷰 크롤링
    def __crawmore(self,sub_info_list:List[Dict],**kwargs):
        sub_drive=Chrome(options=self.chrome_options)
        for sub_info in sub_info_list:
            sub_drive.get(sub_info["url"])

            #별점
            rating=WebDriverWait(sub_drive,3).until(
            presence_of_element_located([By.XPATH, '//*[@role="img"]'])).get_attribute("aria-label")
            #전처리
            rating="".join([term for term in rating if not term.isalpha()]).strip()
            rating=float(rating)

            #음식점 카테고리
            category=WebDriverWait(sub_drive,3).until(
            presence_of_element_located([By.XPATH,"//button[contains(@jsaction,'category')]"])).text

            #정보선택칸
            tabs=WebDriverWait(sub_drive,3).until(
            presence_of_all_elements_located([By.XPATH, '//*[@role="tab"]']))
            tab_select={element.text.strip():element for element in tabs} # 이후 사용할 예정
            
            #하위 정보칸
            info_region=WebDriverWait(sub_drive,3).until(
            presence_of_element_located([By.XPATH, '//*[@role="region"]']))
            #주소
            address=WebDriverWait(info_region,3).until(
            presence_of_element_located([By.XPATH, '//*[@data-item-id="address"]'])).text

            sub_info.update(
                {"address":address,
                "rating":rating,
                "category":category}
            )

            #리뷰 크롤링
            tab_select["리뷰"].click() #리뷰창으로 전환

            #현재 리뷰 개수
            total_review_num=WebDriverWait(sub_drive,3).until(
            presence_of_element_located([By.XPATH,"//div[contains(text(), '리뷰')]"])).text
            #전처리
            total_review_num=int("".join([term for term in total_review_num]))

            #스크롤가능한 바디찾기
            candidate_list=WebDriverWait(sub_drive,3).until(
                 presence_of_all_elements_located([By.XPATH,"//div[@tabindex='-1']"]))
            scroll_body=[element for element in candidate_list 
                         if any(["auto" in element.value_of_css_property("overflow"),
                                 "scroll" in element.value_of_css_property("overflow")])][0]
            

            while True: 
                scroll_body.send_keys(Keys.PAGE_DOWN)

                #리뷰 각각의 객체
                review_box=WebDriverWait(scroll_body,3).until(
                    presence_of_all_elements_located([By.XPATH
                    ,"//div[@data-review-id and contains(@jsaction,'review.out')]"])) 

                if (len(review_box)==total_review_num)or(len(review_box)>self.review_num):
                    break
            
            # 리뷰 돌아가며 더보기 누르기
            for sub_review in review_box:
                sub_drive.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", sub_review)

                try:
                    button=WebDriverWait(sub_review,3).until(
                    presence_of_element_located([By.XPATH,"//button[contains(text(),'자세히')]"]))
                    button.click()
                except: # 리뷰가 짧으면 없음
                    pass             

            
        sub_drive.quit()
        #return sub_info
    
    def craw(self): # 기초정보 수집 -> 1개의 스래딩으로 충분
        self.main_drive.get(self.base_path)
        # 스크롤을 내리는 공간
        main_box=WebDriverWait(self.main_drive,3).until(
        presence_of_element_located([By.XPATH, '//*[@role="feed"]']))

        while True:
            main_box.send_keys(Keys.PAGE_DOWN)

            feed_box=WebDriverWait(self.main_drive,3).until(
                 presence_of_all_elements_located([By.XPATH,"//a[@href and @aria-label]"]))
            

            if len(feed_box)>self.craw_num:
                break
        
        for sub_feed in feed_box:
            #음식점 이름
            restaurant_name=sub_feed.get_attribute("aria-label")
            url=sub_feed.get_attribute("href")

            self.infos.append(
                {"search_term":self.search_term,
                 "restaurant_name":restaurant_name,
                 "url":url}
            )

        self.main_drive.quit()

        thread_list=[]
        m=len(self.infos)//self.thread_num
        for n in range(self.thread_num):
            kw=self.infos[n*m:(n+1)*m]
            sub_thread=Thread(target=self.__crawmore,kwargs=({"sub_info_list":kw}))
            thread_list.append(sub_thread)
            sub_thread.start()
        
        for sub_thread in thread_list:
            sub_thread.join()


In [185]:
test_class=GooglemapCraw("강남역 맛집",20,2,False)

In [ ]:
test_class.infos

In [165]:
url=test_class.infos[-1]["url"]

In [126]:
test_class.close()

In [220]:
test_driver=Chrome()
test_driver.get(url)
#test_driver.get(base_path)

------

In [229]:
feed_box=WebDriverWait(test_driver,3).until(
                 presence_of_all_elements_located([By.XPATH,"//div[@tabindex='-1']"]))

In [233]:
for feed in feed_box:
    print(feed.value_of_css_property("overflow"))
    if "auto" in feed.value_of_css_property("overflow"):
        break

hidden
hidden
visible
visible
visible
visible
visible
visible
hidden
visible
hidden
hidden auto


In [250]:
feed.send_keys(Keys.PAGE_DOWN)

In [236]:
test_box=WebDriverWait(feed,3).until(
                 presence_of_all_elements_located([By.XPATH
                                                   ,"//div[@data-review-id and contains(@jsaction,'review.out')]"]))

In [251]:
test_driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", test_box[1])

In [252]:
but_test=WebDriverWait(test_box[1],3).until(
                    presence_of_element_located([By.XPATH,"//button[contains(text(),'자세히')]"]))

In [254]:
but_test.click()

In [168]:
tabs=WebDriverWait(test_driver,3).until(
            presence_of_all_elements_located([By.XPATH, '//*[@role="tab"]']))
tab_select={element.text.strip():element for element in tabs}

In [255]:
test_driver.quit()

In [171]:
tab_select["리뷰"].click()

In [181]:
test_driver.find_element(By.XPATH,"//div[contains(text(), '리뷰')]")

<selenium.webdriver.remote.webelement.WebElement (session="3b2fad94135727537c2907b051d73ed4", element="f.8B8818B3D0ED11F3C24611D3C1BDB109.d.BBD111B9F2019B07A2B31414B0EE70AE.e.143")>

In [180]:
aa='리뷰 1,167개'
[t for t in aa if t.isdigit()]

['1', '1', '6', '7']

In [ ]:
get_rating=test_driver.find_element(By.CLASS_NAME,"ceNzKf").get_attribute("aria-label")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".jANrlb>ceNzKf"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010504f184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x00000001050479d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000104ab4968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000104af8d50 cxxbridge1$string$len + 368756
4   chromedriver                        0x0000000104b325b4 cxxbridge1$string$len + 604376
5   chromedriver                        0x0000000104aed568 cxxbridge1$string$len + 321676
6   chromedriver                        0x0000000104aee1b8 cxxbridge1$string$len + 324828
7   chromedriver                        0x00000

In [16]:
take_img=WebDriverWait(test_driver,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="img"]']))

In [41]:
res_info=WebDriverWait(test_driver,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="region"]']))

In [ ]:
address=WebDriverWait(res_info,3).until(
    presence_of_element_located([By.XPATH, '//*[@data-item-id="address"]']))

In [45]:
address.get_attribute("aria-label")

'주소: 서울특별시 강남구 강남대로96길 9 '

In [50]:
tabs=WebDriverWait(test_driver,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="tab"]']))

In [52]:
tabs.

'개요'

In [53]:
test_driver.quit()

In [3]:
Main_Drive.quit()

### Restaurant_info

In [88]:
feed_box[0].get_attribute("aria-label")

'바비레드 강남본점'

In [78]:
main_box.get_attribute("role")

'feed'

In [72]:
feed_box[0].get_attribute("aria-label")

'바비레드 강남본점'

In [89]:
Main_Drive.quit()

In [59]:
test_elements=Main_Drive.find_elements(By.CLASS_NAME,"hfpxzc")

MaxRetryError: HTTPConnectionPool(host='localhost', port=57278): Max retries exceeded with url: /session/099672721e87bc05898169d8f5370121/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109ffabb0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [47]:
test_elements[-1].get_attribute("aria-label")

'땀땀'

In [48]:
test_elements[-1].get_attribute("href")

'https://www.google.co.kr/maps/place/%EB%95%80%EB%95%80/data=!4m7!3m6!1s0x357cbba6cab8be71:0x123ebf3924dc0759!8m2!3d37.5003867!4d127.027994!16s%2Fg%2F11hcqrdf1p!19sChIJcb64yqa7fDURWQfcJDm_PhI?authuser=0&hl=ko&rclk=1'

In [49]:
Main_Drive.quit()

In [22]:
# 검색창
feedbox=Main_Drive.find_elements(By.TAG_NAME,"div")

In [25]:
feed_test=[element for element in feedbox if element.get_attribute("role")=="feed"]

In [29]:
feed_test[0].get_attribute("aria-label")

'강남역 맛집에 대한 검색결과'

In [33]:
from selenium.webdriver.common.keys import Keys

for _ in range(10):
    feed_test[0].send_keys(Keys.PAGE_DOWN)

테스트 끝.

In [37]:
Main_Drive.quit()

In [40]:
from threading import Thread

구조 

step 1 : url get

step 2 : url을 통해 세부정보 get